##  Visualizing coefficients on map

In [ ]:
state='NY'
meas='SNWD'

In [2]:
#m=Map(zoom=1)
#m

### Load the required libraries

In [3]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math

import numpy as np
from lib.numpy_pack import packArray,unpackArray
from lib.spark_PCA import computeCov
from lib.computeStatistics import *

from pickle import dump,load


In [4]:
#sc.stop()

In [5]:
%%time
%run lib/startup.py

172.17.0.2
sparkContext= <SparkContext master=local[*] appName=pyspark-shell>



/usr/local/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


    pandas as    pd 	version=1.4.2 	required version>=0.19.2

     numpy as    np 	version=1.21.6 	required version>=1.12.0

   sklearn as    sk 	version=1.0.2 	required version>=0.18.1

module urllib has no version
   pyspark as pyspark 	version=3.2.1 	required version>=2.1.0

ipywidgets as ipywidgets 	version=7.7.0 	required version>=6.0.0

version of ipwidgets= 7.7.0
measurements is a Dataframe (and table) with 12720632 records
stations is a Dataframe (and table) with 119503 records
weather is a Dataframe (and table) which is a join of measurements and stations with 12720632 records
CPU times: user 464 ms, sys: 50.4 ms, total: 514 ms
Wall time: 8.19 s


In [6]:
%%time
ms=[meas]
cms='or\n'.join(['Measurement="%s" '%(m) for m in ms])

##  read all data for state
Query="""
SELECT *
FROM weather
WHERE state="%s" and  measurement='%s'
"""%(state,meas)
print(Query)

weather_df=sqlContext.sql(Query) #weather_df contains the information for the state
print('number of rows in result=',weather_df.count())
weather_df.show(1)


SELECT *
FROM weather
WHERE state="NY" and  measurement='SNWD'

number of rows in result= 27002
+-----------+-----------+----+--------------------+--------+---------+---------+----------+----------------+-----+-------------+
|    Station|Measurement|Year|              Values|latitude|longitude|elevation|dist2coast|            name|state|      country|
+-----------+-----------+----+--------------------+--------+---------+---------+----------+----------------+-----+-------------+
|US1NYDT0024|       SNWD|2022|[00 00 00 00 00 0...| 42.0097| -73.8642|     65.8|    108.25|RED HOOK 1.2 NNE|   NY|United States|
+-----------+-----------+----+--------------------+--------+---------+---------+----------+----------------+-----+-------------+
only showing top 1 row

CPU times: user 3.27 ms, sys: 168 µs, total: 3.44 ms
Wall time: 1.42 s


In [7]:
pkl_filename=parquet_root+'/weather-statistics/'+state+'-'+','.join(ms)+'.pkl'
!ls $pkl_filename 

/datasets/cs255-sp22-a00-public//weather-statistics/NY-SNWD.pkl


In [8]:
stat=load_or_compute_statistics(pkl_filename,weather_df,ms)
STAT=stat
STAT.keys()

precomputed statistics file exists


dict_keys(['SNWD'])

### compute spectral decomposition

In [9]:
%%time
#get mean and eigenvectors for measurement m
EigVec=STAT[meas]['eigvec']
Mean=STAT[meas]['Mean']
EigVec.shape

k=5
decomposition=decompose_dataframe(sqlContext,weather_df,EigVec[:,:k],Mean).cache() # Make it possible to generate only first k coefficients.

CPU times: user 11.5 ms, sys: 6.69 ms, total: 18.2 ms
Wall time: 760 ms


In [10]:
%%time
decomposition.show(1)
decomposition.count()

+-----------+-----------+----+--------------------+--------+---------+---------+----------+----------------+-----+-------------+---------+-----------------+------------------+-------------------+-------------------+------------------+-------------------+-----------------+-------------------+------------------+-----------------+-----------------+
|    Station|Measurement|Year|              Values|latitude|longitude|elevation|dist2coast|            name|state|      country|total_var|         res_mean|             res_1|            coeff_1|              res_2|           coeff_2|              res_3|          coeff_3|              res_4|           coeff_4|            res_5|          coeff_5|
+-----------+-----------+----+--------------------+--------+---------+---------+----------+----------------+-----+-------------+---------+-----------------+------------------+-------------------+-------------------+------------------+-------------------+-----------------+-------------------+------------

27002

In [17]:
## To save the result you need to give a directory for which you have write permissions
decomp_parquet=parquet_root+'weather-statistics/'+state+'-'+meas+'.parquet'
print('saving to',decomp_parquet)
!rm -rf $decomp_parquet
decomposition.write.parquet(decomp_parquet)

saving to /datasets/cs255-sp22-a00-public/weather-statistics/NY-SNWD.parquet


## Compute the count and average of `coeff_i` for each station.

In [ ]:
feature='coeff_1'
df1 = decomposition.select('station','latitude','longitude','elevation','dist2coast',feature)
df1.show(2)

In [ ]:
df2=df1.groupby(['station','latitude','longitude','elevation','dist2coast'])\
.agg({"station": "count", feature: "mean"})
#df2=df1.groupby(['station']).agg({"station": "count", feature: "mean"})
pdf=df2.toPandas()
pdf.sort_values(by=['station'],inplace=True)
pdf.head(5)

In [ ]:
# Find long/lat bounding box
max_long=pdf['longitude'].max()
min_long=pdf['longitude'].min()
max_lat=pdf['latitude'].max()
min_lat=pdf['latitude'].min()


In [ ]:
#define a mapping from the range of the value to hex colors.
from matplotlib.colors import rgb2hex
_avg='avg(%s)'%feature
_min=pdf[_avg].min()
_max=pdf[_avg].max()
_min,_max

In [ ]:
import pylab as plt
cmap=plt.get_cmap('jet')
def get_color(val):
    x=(val-_min)/(_max-_min)
    return(rgb2hex(cmap(x)[:3]))

## Map
* Each circle is centered at a station.  
* The area of the circle corresponds to the number of years SNWD was recorded at the station.  
* The color fill of the circle corresponds to the value of `avg(coeff_1)` defined by color-bar.

In [ ]:
import ipyleaflet
#%pylab inline

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

In [ ]:
#def color_bar(_min,_max)
import pylab as plt
from matplotlib.pyplot import xticks,yticks
#figure(figsize=[10,1])
ax = plt.subplot(111)
vals=cmap(np.arange(0,1,0.005))[:,:3]
vals3=np.stack([vals]*10)
vals3.shape
ax.imshow(vals3);

midpoint=200.*-_min/(_max-_min)
xticks((0,midpoint,200),["%4.1f"%v for v in (_min,0.,_max)])
yticks(());


center = [(min_lat+max_lat)/2, (min_long+max_long)/2]


zoom = 6

m = Map(default_tiles=TileLayer(opacity=1.0), center=center, zoom=zoom)
# basemap=basemaps.OpenTopoMap,

r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
circles = []
for index,row in pdf.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    _count=row['count(station)']
    _coef=row[_avg]
    # taking sqrt of count so that the  area of the circle corresponds to the count
    c = Circle(location=(_lat,_long), radius=int(1200*np.sqrt(_count+0.0)), weight=1,
            color='#F00', opacity=0.8, fill_opacity=0.4,
            fill_color=get_color(_coef))
    circles.append(c)
    m.add_layer(c)
m    

### Is coef_1 related to elevation ?

In [ ]:
pdf.plot.scatter(x='elevation',y='avg(%s)'%feature,figsize=(8,6));
plt.grid()
#plt.xlim([-100,1000])

In [ ]:
pdf.plot.scatter(x='latitude',y='avg(%s)'%feature,figsize=(8,6));
plt.grid()
plt.xlim([40,45.2])

In [ ]:
pdf.plot.scatter(x='elevation',y='latitude',figsize=(8,6));
plt.grid()
#plt.xlim([0,1000])

## Summary
* We saw how to use `ipyLeaflet` to present data on top of maps.
* We saw that in NY state, most of the snow accumulation is in the Adirondacks.
* Snow accumulation increases with elevation, but the relationship is weak: locations with elevation 400-600 meters have widely varying accumulations of snow.

## See you next time